Yes. Import the model and call predict directly:

```python
from pathlib import Path
from serverless_tuner_middleware.model import fit_regression_model, REGRESSION_MODEL

# Use the pre-fit default model
lat_ms = REGRESSION_MODEL.predict(
    "http", "ea1->ea1", "p50", payload_bytes=10*1024, rate_rps=1.0
)

# Or fit from a custom CSV directory, then predict
m = fit_regression_model(Path("middleware/serverless_tuner_middleware/csv"))
lat_ms = m.predict("pubsub", "ea1->we1", "p90", payload_bytes=200*1024, rate_rps=2.0)
print(lat_ms)
```

Unified form with a flexible rate term: lat_q = c_floor_q + a_q / (k_q + rate_rps) + d_rate_q * rate_rps + b_size_q * payload_bytes. Constrain a_q, k_q, b_size_q ≥ 0, but let d_rate_q be signed so HTTP can slope slightly up while Pub/Sub gets the 1/(k+rate) benefit.


In [1]:
from middleware.serverless_tuner_middleware.model import REGRESSION_MODEL
for k,v in REGRESSION_MODEL.coeffs.items():
    print(k, v)

('pubsub', 'us-east1->us-east1', 'p50') Coeffs(a_inv_rate=5.865408051228487, b_size=2.989880066140614e-05, c_floor=2.1252062873468986, d_rate=7.627377713099122, k_shift=0.01)
('http', 'us-east1->us-west1', 'p50') Coeffs(a_inv_rate=10.405549002836011, b_size=0.0004141352149209306, c_floor=4.942788426033644, d_rate=19.523151576519012, k_shift=0.01)
('pubsub', 'us-east1->us-west1', 'p50') Coeffs(a_inv_rate=10.947183770090419, b_size=0.00045125855403260756, c_floor=3.8441811528053904, d_rate=13.610772460699081, k_shift=0.01)
('http', 'us-east1->us-east1', 'p50') Coeffs(a_inv_rate=6.187576644196875, b_size=7.969001825315003e-06, c_floor=2.901354688456081, d_rate=11.41594912391156, k_shift=0.01)
('pubsub', 'us-east1->us-east1', 'p90') Coeffs(a_inv_rate=8.842123539015311, b_size=3.260264670580748e-05, c_floor=3.4713737289580138, d_rate=12.865825617685914, k_shift=0.01)
('http', 'us-east1->us-west1', 'p90') Coeffs(a_inv_rate=11.621247962360798, b_size=0.00044946679383717637, c_floor=5.58135086

In [6]:
from collections import defaultdict
from pathlib import Path
from middleware.serverless_tuner_middleware.logs import parse_events_from_lines
from middleware.serverless_tuner_middleware.stats import (
    compute_edge_samples, aggregate_edge_stats,
    compute_node_samples, aggregate_node_stats,
)
from middleware.serverless_tuner_middleware.critical_path import edge_key
from middleware.serverless_tuner_middleware.rewrite import _infer_region_pair
from middleware.serverless_tuner_middleware.model import REGRESSION_MODEL
from middleware.serverless_tuner_middleware.config import load_config

LOGS = Path("results/tts/newlogs_fixed.ndjson")
CFG = load_config("results/tts/invoker_config.ndjson")

with LOGS.open() as f:
    sends, recvs = parse_events_from_lines(f)

edge_stats = aggregate_edge_stats(compute_edge_samples(sends, recvs))
node_stats = aggregate_node_stats(compute_node_samples(sends, recvs))

payloads = defaultdict(list)
ts_map = defaultdict(list)
for s in sends:
    payloads[edge_key(s)].append(s.payload_size)
    ts_map[edge_key(s)].append(s.ts_ms)

rates = {}
for ek, ts in ts_map.items():
    dur = (max(ts) - min(ts)) / 1000 if len(ts) > 1 else 0
    rates[ek] = len(ts) / dur if dur > 0 else 0.0

print("edge_key | payload_avg/min/max (B) | rate_rps | observed_p50_ms | model_p50_http | model_p50_pubsub")
for e in CFG.edges:
    ek = edge_key(e)
    region = _infer_region_pair(e)
    obs = edge_stats.get((ek, "http")) or edge_stats.get((ek, "pubsub"))
    obs_p50 = obs.p50 if obs else None

    sizes = payloads.get(ek, [])
    avg_payload = sum(sizes) / len(sizes) if sizes else None
    min_payload = min(sizes) if sizes else None
    max_payload = max(sizes) if sizes else None
    rate = rates.get(ek, 0.0)

    http_pred = pubsub_pred = None
    if region and avg_payload is not None:
        http_pred = REGRESSION_MODEL.predict("http", region, "p50", payload_bytes=avg_payload, rate_rps=rate)
        pubsub_pred = REGRESSION_MODEL.predict("pubsub", region, "p50", payload_bytes=avg_payload, rate_rps=rate)

    print(f"{ek} | {avg_payload} / {min_payload} / {max_payload} | {rate:.4f} | {obs_p50} | {http_pred} | {pubsub_pred}")

print("\nnode runtimes (p50):")
for fn, stat in node_stats.items():
    print(f"{fn}: p50={stat.p50} ms count={stat.count}")


edge_key | payload_avg/min/max (B) | rate_rps | observed_p50_ms | model_p50_http | model_p50_pubsub
get_input:entry_point:0->text_2_speech:get_input_0_0:1 | 17047.70491803279 / 17009 / 17048 | 0.0306 | 111.5 | 155.85438223523772 | 147.39766696501428
get_input:entry_point:0->profanity:get_input_0_1:2 | 17035.70108695652 / 16997 / 17036 | 0.0307 | 111.0 | 155.2308710170274 | 146.80581804664044
text_2_speech:get_input_0_0:1->encoding:text_2_speech_1_0:3 | 17050.083333333332 / 17012 / 17051 | 0.0439 | 81.5 | 118.24435607142732 | 111.70325997040051
profanity:get_input_0_1:2->censor:sync: | 16896.333333333332 / 16864 / 16903 | 0.0054 | 65.0 | 406.1243411300946 | 384.7140251167854
encoding:text_2_speech_1_0:3->censor:sync: | 16902.904761904763 / 16902 / 16903 | 0.1418 | 131.0 | 45.40589285536856 | 42.34128855409308

node runtimes (p50):
encoding: p50=1632.0 ms count=33
text_2_speech: p50=7555.0 ms count=29
profanity: p50=19225.0 ms count=5


In [5]:
from collections import defaultdict
from pathlib import Path
from middleware.serverless_tuner_middleware.logs import parse_events_from_lines

with Path("results/tts/newlogs_fixed.ndjson").open() as f:
    sends, recvs = parse_events_from_lines(f)

recvs_by_fn = defaultdict(set)
sends_by_fn = defaultdict(set)
for r in recvs:
    recvs_by_fn[r.fn_name].add(r.run_id)   # run_id only
for s in sends:
    sends_by_fn[s.from_fn].add(s.run_id)   # run_id only

for fn in recvs_by_fn:
    missing = recvs_by_fn[fn] - sends_by_fn[fn]
    print(f"{fn}: recvs={len(recvs_by_fn[fn])}, sends={len(sends_by_fn[fn])}, missing_runtime={len(missing)}")


# Edge sample counts
edge_counts = defaultdict(int)
for s in sends:
    edge_counts[edge_key(s)] += 1
print("\nEdge send counts:")
for ek, cnt in edge_counts.items():
    print(ek, cnt)

text_2_speech: recvs=174, sends=48, missing_runtime=145
profanity: recvs=193, sends=5, missing_runtime=188
censor: recvs=45, sends=0, missing_runtime=45
encoding: recvs=37, sends=43, missing_runtime=4

Edge send counts:
get_input:entry_point:0->profanity:get_input_0_1:2 184
get_input:entry_point:0->text_2_speech:get_input_0_0:1 183
profanity:get_input_0_1:2->censor:sync: 6
encoding:text_2_speech_1_0:3->censor:sync: 42
text_2_speech:get_input_0_0:1->encoding:text_2_speech_1_0:3 48


In [8]:
from collections import Counter
from pathlib import Path
from middleware.serverless_tuner_middleware.logs import parse_events_from_lines
from middleware.serverless_tuner_middleware.critical_path import edge_key

with Path("results/tts/logs.ndjson").open() as f:
    sends, recvs = parse_events_from_lines(f)

send_pairs = Counter((s.run_id, s.taint, s.to_fn) for s in sends if s.from_fn=="profanity")
recv_pairs = Counter((r.run_id, r.taint, r.fn_name) for r in recvs if r.fn_name=="censor")
print("matched sends->recvs:", sum(min(send_pairs[p], recv_pairs.get(p,0)) for p in send_pairs))
print("total sends:", len(send_pairs), "total recvs:", len(recv_pairs))


matched sends->recvs: 21
total sends: 22 total recvs: 121


In [9]:
from pathlib import Path
from collections import Counter
from middleware.serverless_tuner_middleware.logs import parse_events_from_lines
from middleware.serverless_tuner_middleware.critical_path import edge_key

with Path("results/tts/logs.ndjson").open() as f:
    sends, _ = parse_events_from_lines(f)

edges = Counter(edge_key(s) for s in sends if s.from_fn == "encoding")
print(edges)


Counter({'profanity:get_input_0_1:2->censor:sync:': 107})


In [4]:
from collections import Counter
from pathlib import Path
from middleware.serverless_tuner_middleware.logs import parse_events_from_lines
from middleware.serverless_tuner_middleware.critical_path import edge_key

with Path("results/tts/newlogs_fixed.ndjson").open() as f:
    sends, _ = parse_events_from_lines(f)

by_edge = Counter(edge_key(s) for s in sends)
print(by_edge.get("encoding:text_2_speech_1_0:3->censor:sync:"))  # likely 0
print(by_edge.get("profanity:get_input_0_1:2->censor:sync:"))      # 107


42
6


In [2]:
from collections import Counter
from pathlib import Path
from middleware.serverless_tuner_middleware.logs import parse_events_from_lines
from middleware.serverless_tuner_middleware.critical_path import edge_key

with Path("results/tts/newlogs_fixed.ndjson").open() as f:
    sends, _ = parse_events_from_lines(f)

# All edges emitted by encoding
enc_edges = Counter(edge_key(s) for s in sends if s.from_fn == "encoding")
print("encoding sends:", enc_edges)

# All edges to censor (any source)
censor_edges = Counter(edge_key(s) for s in sends if s.to_fn == "censor")
print("to censor:", censor_edges)


encoding sends: Counter({'encoding:text_2_speech_1_0:3->censor:sync:': 42, 'profanity:get_input_0_1:2->censor:sync:': 1})
to censor: Counter({'encoding:text_2_speech_1_0:3->censor:sync:': 42, 'profanity:get_input_0_1:2->censor:sync:': 6})


In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
from middleware.serverless_tuner_middleware import constants

# Summary: raw vs trimmed, plus share of samples
total_rows = len(df)
summaries = []
for ek, g in df.groupby("edge_key"):
    raw_p50 = g["lat_ms"].median()
    raw_p90 = g["lat_ms"].quantile(0.9)
    trimmed = g[~g["is_outlier"]]
    trim_p50 = trimmed["lat_ms"].median() if not trimmed.empty else None
    trim_p90 = trimmed["lat_ms"].quantile(0.9) if not trimmed.empty else None
    share = len(g) / total_rows if total_rows else 0.0
    summaries.append({
        "edge_key": ek,
        "count": len(g),
        "share": share,
        "outliers": int(g["is_outlier"].sum()),
        "raw_p50": raw_p50,
        "raw_p90": raw_p90,
        "trim_p50": trim_p50,
        "trim_p90": trim_p90,
    })
print("Per-edge latency (ms), raw vs trimmed outlier filter and share:\n")
for row in summaries:
    trim_p50 = f"{row['trim_p50']:.2f}" if row["trim_p50"] is not None else "NA"
    trim_p90 = f"{row['trim_p90']:.2f}" if row["trim_p90"] is not None else "NA"
    print(
        f"{row['edge_key']}: count={row['count']} share={row['share']*100:.1f}% outliers={row['outliers']} "
        f"raw_p50={row['raw_p50']:.2f} raw_p90={row['raw_p90']:.2f} "
        f"trim_p50={trim_p50} trim_p90={trim_p90}"
    )

# Per-run bottleneck frequency (using trimmed data)
trimmed_df = df[~df["is_outlier"]]
if not trimmed_df.empty:
    idx = trimmed_df.groupby("run_id")["lat_ms"].idxmax()
    winners = trimmed_df.loc[idx, "edge_key"].value_counts()
    print("\nPer-run bottleneck frequency (trimmed):")
    for ek, cnt in winners.items():
        print(f"{ek}: {cnt}")
else:
    print("\nNo trimmed samples to compute bottlenecks.")

# Per-run bottleneck frequency restricted to sync edges only
sync_mask = trimmed_df["edge_key"].str.contains(":sync:") if not trimmed_df.empty else pd.Series([], dtype=bool)
trimmed_sync = trimmed_df[sync_mask]
if not trimmed_sync.empty:
    idx_sync = trimmed_sync.groupby("run_id")["lat_ms"].idxmax()
    winners_sync = trimmed_sync.loc[idx_sync, "edge_key"].value_counts()
    sync_run_total = trimmed_sync["run_id"].nunique()
    threshold = getattr(constants, "SYNC_BOTTLENECK_RUN_SHARE_THRESHOLD", 0.0)
    print(
        "\nPer-run bottleneck frequency (trimmed, sync-only): "
        f"threshold={threshold*100:.0f}% of runs"
    )
    for ek, cnt in winners_sync.items():
        run_share = cnt / sync_run_total if sync_run_total else 0.0
        sample_count = len(trimmed_sync[trimmed_sync["edge_key"] == ek])
        flag = " ✅" if run_share >= threshold else ""
        print(f"{ek}: {cnt}/{sync_run_total} runs ({run_share*100:.1f}%) samples={sample_count}{flag}")

    share_df = winners_sync.rename("runs").reset_index().rename(columns={"index": "edge_key"})
    share_df["run_share"] = share_df["runs"] / sync_run_total if sync_run_total else 0.0
    fig_share = px.bar(
        share_df,
        x="edge_key",
        y="run_share",
        text=share_df["run_share"].map(lambda x: f"{x*100:.1f}%"),
        title="Sync-edge bottleneck run share (trimmed)",
    )
    fig_share.add_hline(
        y=threshold,
        line_dash="dash",
        line_color="red",
        annotation_text="threshold",
        annotation_position="top left",
    )
    fig_share.update_yaxes(tickformat=".0%")
    fig_share.update_layout(margin=dict(t=60, r=20))
    fig_share.show()
else:
    print("\nNo sync-edge samples to compute sync-only bottlenecks.")

# Scatterplot of latencies with outliers highlighted; separate facets per edge
df["label"] = np.where(df["is_outlier"], "outlier", "inlier")
fig = px.scatter(
    df,
    x="send_ts",
    y="lat_ms",
    color="label",
    facet_row="edge_key",
    facet_row_spacing=0.08,
    hover_data=["edge_key", "mechanism", "payload_size", "idle_gap_s"],
    title="Edge latencies (ms) with outlier flag",
    height=max(400, 200 * df["edge_key"].nunique()),
)
fig.update_yaxes(matches=None)
# Move facet labels above each subplot, horizontal
fig.for_each_annotation(
    lambda a: a.update(
        text=a.text.split("=")[-1],
        x=0.5,
        xanchor="center",
        y=a.y + 0.06,
        yanchor="bottom",
        textangle=0,
    )
)
fig.update_layout(margin=dict(r=20, t=80))
fig.show()

Per-edge latency (ms), raw vs trimmed outlier filter and share:

encoding:text_2_speech_1_0:3->censor:sync:: count=40 share=9.7% outliers=2 raw_p50=131.00 raw_p90=467.50 trim_p50=126.50 trim_p90=442.70
get_input:entry_point:0->profanity:get_input_0_1:2: count=176 share=42.6% outliers=9 raw_p50=111.00 raw_p90=206.50 trim_p50=109.00 trim_p90=139.00
get_input:entry_point:0->text_2_speech:get_input_0_0:1: count=156 share=37.8% outliers=8 raw_p50=111.50 raw_p90=156.50 trim_p50=109.50 trim_p90=137.30
profanity:get_input_0_1:2->censor:sync:: count=5 share=1.2% outliers=1 raw_p50=65.00 raw_p90=70.80 trim_p50=65.00 trim_p90=65.70
text_2_speech:get_input_0_0:1->encoding:text_2_speech_1_0:3: count=36 share=8.7% outliers=2 raw_p50=81.50 raw_p90=740.50 trim_p50=79.50 trim_p90=660.70

Per-run bottleneck frequency (trimmed):
get_input:entry_point:0->profanity:get_input_0_1:2: 91
get_input:entry_point:0->text_2_speech:get_input_0_0:1: 79
encoding:text_2_speech_1_0:3->censor:sync:: 16
text_2_speech:get

Outlier/idle-gap trimming notes:
- Samples per edge match send↔recv on (run_id, taint).
- Outlier rule: drop points above max(P95, P75 + 1.5×IQR) per edge.
- Idle-gap is computed per edge (seconds since previous send); inspect in hover.
- The scatter facets per edge highlight outliers in red.

In [15]:
from collections import defaultdict
from pathlib import Path
import pandas as pd
from middleware.serverless_tuner_middleware import constants
from middleware.serverless_tuner_middleware.logs import parse_events_from_lines
from middleware.serverless_tuner_middleware.stats import (
    compute_edge_samples,
    aggregate_edge_stats,
    compute_node_samples,
    aggregate_node_stats,
)
from middleware.serverless_tuner_middleware.critical_path import build_dag, critical_path_from_stats, edge_key
from middleware.serverless_tuner_middleware.rewrite import _infer_region_pair, _mechanism_cost_or_model, _weight_for_edge, _sync_bottleneck_edges
from middleware.serverless_tuner_middleware.model import REGRESSION_MODEL
from middleware.serverless_tuner_middleware.config import load_config

LOGS = Path("results/tts/newlogs_fixed.ndjson")
CFG = load_config("results/tts/invoker_config.ndjson")

with LOGS.open() as f:
    sends, recvs = parse_events_from_lines(f)

edge_samples = compute_edge_samples(sends, recvs)
node_samples = compute_node_samples(sends, recvs)
edge_stats = aggregate_edge_stats(edge_samples)
node_stats = aggregate_node_stats(node_samples)

# Edge context: payload stats and inferred rate per edge
payloads = defaultdict(list)
ts_map = defaultdict(list)
for s in sends:
    payloads[edge_key(s)].append(s.payload_size)
    ts_map[edge_key(s)].append(s.ts_ms)

rates = {}
for ek, ts in ts_map.items():
    dur = (max(ts) - min(ts)) / 1000 if len(ts) > 1 else 0
    rates[ek] = len(ts) / dur if dur > 0 else 0.0

print("edge_key | payload_avg/min/max (B) | rate_rps | observed_p50_ms | model_p50_http | model_p50_pubsub")
for e in CFG.edges:
    ek = edge_key(e)
    region = _infer_region_pair(e)
    obs = edge_stats.get((ek, "http")) or edge_stats.get((ek, "pubsub"))
    obs_p50 = obs.p50 if obs else None

    sizes = payloads.get(ek, [])
    avg_payload = int(sum(sizes) / len(sizes)) if sizes else None
    min_payload = min(sizes) if sizes else None
    max_payload = max(sizes) if sizes else None
    rate = rates.get(ek, 0.0)

    http_pred = pubsub_pred = None
    if region and avg_payload is not None:
        http_pred = REGRESSION_MODEL.predict("http", region, "p50", payload_bytes=avg_payload, rate_rps=rate)
        pubsub_pred = REGRESSION_MODEL.predict("pubsub", region, "p50", payload_bytes=avg_payload, rate_rps=rate)

    print(f"{ek} | {avg_payload} / {min_payload} / {max_payload} | {rate:.4f} | {obs_p50} | {http_pred} | {pubsub_pred}")

print("\nnode runtimes (p50):")
for fn, stat in node_stats.items():
    print(f"{fn}: p50={stat.p50} ms count={stat.count}")

# Compute sync bottleneck flags using the same thresholds as rewrite
sync_info, flagged_sync = _sync_bottleneck_edges(
    edge_samples,
    min_samples=constants.SYNC_CRITICAL_MIN_SAMPLES,
    run_share_threshold=constants.SYNC_BOTTLENECK_RUN_SHARE_THRESHOLD,
)
print("\nSync-edge bottleneck run share (thresholded):")
if sync_info:
    for ek, (share, cnt) in sync_info.items():
        mark = " ✅" if ek in flagged_sync else ""
        print(f"{ek}: run_share={share*100:.1f}% samples={cnt}{mark}")
else:
    print("No sync samples")

# Build edge weights for critical path computation (p50 observed or model fallback)
edge_context = {}
edge_weights = {}
for e in CFG.edges:
    ek = edge_key(e)
    sizes = payloads.get(ek, [])
    avg_payload = int(sum(sizes) / len(sizes)) if sizes else None
    ts = ts_map.get(ek, [])
    duration_s = (max(ts) - min(ts)) / 1000.0 if len(ts) > 1 else 0.0
    rate_rps = (len(ts) / duration_s) if duration_s > 0 else None
    region = _infer_region_pair(e)
    edge_context[ek] = (region, avg_payload, rate_rps)

    edge_weights[ek] = _weight_for_edge(
        e,
        edge_stats,
        region_pair=region,
        payload_size_bytes=avg_payload,
        rate_rps=rate_rps,
    )

dag = build_dag(CFG)

# Path with node weights (what the greedy optimizer uses today)
node_weights = {fn: s.p50 for fn, s in node_stats.items()}
path_cost_with_nodes, path_nodes_with_nodes = critical_path_from_stats(
    dag, edge_stats=edge_weights, node_stats=node_weights
)

# Edge-only path (node weights ignored) to align with sync-bottleneck intent
path_cost_edges_only, path_nodes_edges_only = critical_path_from_stats(
    dag, edge_stats=edge_weights, node_stats={}
),

print(f"\nCritical path with node weights: {path_nodes_with_nodes} (cost={path_cost_with_nodes:.2f} ms)")
print(f"Critical path edges-only:       {path_nodes_edges_only} (cost={path_cost_edges_only:.2f} ms)")

# Evaluate flip gains on the edge-only critical path (to focus on transport + sync flags)
path_edges = list(zip(path_nodes_edges_only, path_nodes_edges_only[1:]))
rows = []
for frm, to in path_edges:
    edge = next(e for e in CFG.edges if e.from_fn == frm and e.to_fn == to)
    ek = edge_key(edge)
    ctx = edge_context.get(ek)
    region = ctx[0] if ctx else None
    size_bytes = ctx[1] if ctx else None
    rate_rps = ctx[2] if ctx else None

    current = edge.strategy.lower()
    current_cost = _mechanism_cost_or_model(edge, current, edge_stats, region_pair=region, payload_size_bytes=size_bytes, rate_rps=rate_rps)
    if current_cost is None:
        current_cost = _weight_for_edge(edge, edge_stats, region_pair=region, payload_size_bytes=size_bytes, rate_rps=rate_rps)

    http_cost = _mechanism_cost_or_model(edge, "http", edge_stats, region_pair=region, payload_size_bytes=size_bytes, rate_rps=rate_rps)
    pubsub_cost = _mechanism_cost_or_model(edge, "pubsub", edge_stats, region_pair=region, payload_size_bytes=size_bytes, rate_rps=rate_rps)

    best_cost = None
    best_mech = None
    for cost, mech in ((http_cost, "http"), (pubsub_cost, "pubsub")):
        if cost is None:
            continue
        if best_cost is None or cost < best_cost:
            best_cost = cost
            best_mech = mech

    gain = (current_cost if current_cost is not None else best_cost) - best_cost if best_cost is not None else 0.0
    rows.append({
        "edge_key": ek,
        "current": current,
        "best_mech": best_mech,
        "gain_ms": gain,
        "current_cost_ms": current_cost,
        "best_cost_ms": best_cost,
        "flagged_sync": ek in flagged_sync,
    })

if rows:
    df_flips = pd.DataFrame(rows)
    print("\nEdge-only critical-path edges and flip gains:")
    print(df_flips.to_string(index=False))

    flip_candidates = df_flips[(df_flips["best_mech"].notna()) & (df_flips["best_mech"] != df_flips["current"]) & (df_flips["gain_ms"] > constants.GAIN_THRESHOLD_MS)]
    print("\nFlip candidates above gain floor (edge-only path, sync flags shown):")
    if flip_candidates.empty:
        print("None – gain floor not met.")
    else:
        print(flip_candidates.to_string(index=False))
else:
    print("No critical path edges found.")

edge_key | payload_avg/min/max (B) | rate_rps | observed_p50_ms | model_p50_http | model_p50_pubsub
get_input:entry_point:0->text_2_speech:get_input_0_0:1 | 17047 / 17009 / 17048 | 0.0306 | 111.5 | 155.85437661774463 | 147.39764588881053
get_input:entry_point:0->profanity:get_input_0_1:2 | 17035 / 16997 / 17036 | 0.0307 | 111.0 | 155.23086543006417 | 146.80579708498126
text_2_speech:get_input_0_0:1->encoding:text_2_speech_1_0:3 | 17050 / 17012 / 17051 | 0.0439 | 81.5 | 118.24435540734383 | 111.70325747883379
profanity:get_input_0_1:2->censor:sync: | 16896 / 16864 / 16903 | 0.0054 | 65.0 | 406.12433847376064 | 384.71401515051855
encoding:text_2_speech_1_0:3->censor:sync: | 16902 / 16902 / 16903 | 0.1418 | 131.0 | 45.40588564531929 | 42.341261502797245

node runtimes (p50):
encoding: p50=1632.0 ms count=33
text_2_speech: p50=7555.0 ms count=29
profanity: p50=19225.0 ms count=5

Sync-edge bottleneck run share (thresholded):
profanity:get_input_0_1:2->censor:sync:: run_share=11.1% samples=

ValueError: not enough values to unpack (expected 2, got 1)